In [130]:
from binance.client import Client
import pandas as pd
import numpy as np
import math
import time
import statistics
from coin_data import data
from config import api_key, api_secret
import monedas
import json

In [145]:
l = []

with open("./coin_data.json", "r", encoding='utf-8') as f:
    d = json.load(f)
print(len(d))
print(type(d))

for k in d.keys():
    l.append(k)
# aqui extraje los ticker symbols
print(l[138])

139
<class 'dict'>
OPUSDT


In [98]:
print(data)

{'ROSEUSDT': {'i1': 0.01, 'i2': 0.001, 'minAmntUsd': 5.1, 'price_decimal': 5, 'qty_decimal': 0}, 'MANAUSDT': {'i1': 0.1, 'i2': 0.01, 'minAmntUsd': 5.31, 'price_decimal': 4, 'qty_decimal': 0}, 'ADAUSDT': {'i1': 0.01, 'i2': 0.001, 'minAmntUsd': 5.37, 'price_decimal': 4, 'qty_decimal': 0}, 'BTCUSDT': {'i1': 100, 'i2': 50, 'minAmntUsd': 24, 'price_decimal': 1, 'qty_decimal': 3}, 'MATICUSDT': {'i1': 0.01, 'i2': 0.001, 'minAmntUsd': 5.6, 'price_decimal': 4, 'qty_decimal': 0}, 'ETHUSDT': {'i1': 100, 'i2': 50, 'minAmntUsd': 5.5, 'price_decimal': 2, 'qty_decimal': 3}, 'BCHUSDT': {'i1': 10, 'i2': 1, 'minAmntUsd': 5.1, 'price_decimal': 2, 'qty_decimal': 3}, 'XRPUSDT': {'i1': 0.01, 'i2': 0.001, 'minAmntUsd': 5.1, 'price_decimal': 4, 'qty_decimal': 1}, 'EOSUSDT': {'i1': 0.1, 'i2': 0.01, 'minAmntUsd': 5.1, 'price_decimal': 3, 'qty_decimal': 1}, 'LTCUSDT': {'i1': 1, 'i2': 0.1, 'minAmntUsd': 5.1, 'price_decimal': 2, 'qty_decimal': 3}, 'TRXUSDT': {'i1': 0.001, 'i2': 0.0001, 'minAmntUsd': 5.1, 'price_de

In [99]:
client = Client(api_key=api_key, api_secret= api_secret)
ticker = 'STORJUSDT'

# 1. Extraer datos de la moneda

- Traemos toda la info de todas las monedas de futuros de binance

In [100]:
info = client.futures_exchange_info()

- Buscamos la moneda que necesitamos
- Y extraemos la información que creamos necesaria
- Esto lo deberíamos ejecutar 1 sola vez para todas las monedas y guardarlo para solo consultarlo como diccionario y no extraer la información cada vez
- Aunque los datos traen la precisión para el precio, en algunas monedas está mal. Por lo tanto, se crea la función que retorna la cantidad de decimales a través de otra propiedad listada

In [101]:
def get_precision(texto:str) -> int:
    for i in range(2, len(texto)):
        if texto[i] == '1':
            return i-1

In [102]:
for d in info['symbols']:
    if d['pair']==ticker:
        print(d)
        print("")
        print(f"Moneda: {d['pair']}\nSubtipo: {d['underlyingSubType']}\nPrecisión Precio: {get_precision(d['filters'][0]['tickSize'])} decimales")


{'symbol': 'STORJUSDT', 'pair': 'STORJUSDT', 'contractType': 'PERPETUAL', 'deliveryDate': 4133404800000, 'onboardDate': 1569398400000, 'status': 'TRADING', 'maintMarginPercent': '2.5000', 'requiredMarginPercent': '5.0000', 'baseAsset': 'STORJ', 'quoteAsset': 'USDT', 'marginAsset': 'USDT', 'pricePrecision': 4, 'quantityPrecision': 0, 'baseAssetPrecision': 8, 'quotePrecision': 8, 'underlyingType': 'COIN', 'underlyingSubType': ['Storage'], 'settlePlan': 0, 'triggerProtect': '0.0500', 'liquidationFee': '0.025000', 'marketTakeBound': '0.10', 'filters': [{'minPrice': '0.0190', 'maxPrice': '121', 'filterType': 'PRICE_FILTER', 'tickSize': '0.0001'}, {'stepSize': '1', 'filterType': 'LOT_SIZE', 'maxQty': '10000000', 'minQty': '1'}, {'stepSize': '1', 'filterType': 'MARKET_LOT_SIZE', 'maxQty': '20000', 'minQty': '1'}, {'limit': 200, 'filterType': 'MAX_NUM_ORDERS'}, {'limit': 10, 'filterType': 'MAX_NUM_ALGO_ORDERS'}, {'notional': '5', 'filterType': 'MIN_NOTIONAL'}, {'multiplierDown': '0.9000', 'mul

# 2. Obtener Shock Points

# a. Libro de ordenes

- Vamos a obtener el libro de órdenes de la moneda
- Luego separamos las ventas de las compras y las colocamos en un dataframe para mejor organización
- Como nota, lo que trae el libro no es la cantidad total de monedas por bloque sino el total de USDT
- En el caso de las ventas, para mostrarlo como lo vemos en Binance (mayor a menor) ordenamis descebdentemente

In [103]:
order_book = client.futures_order_book(symbol=ticker, limit=1000)
#order_book

In [104]:
ventas = order_book['asks']
ventas = pd.DataFrame(ventas, columns=['precio','total_usdt']).sort_index(0, ascending=False)
ventas

/tmp/ipykernel_697/4237633481.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  ventas = pd.DataFrame(ventas, columns=['precio','total_usdt']).sort_index(0, ascending=False)


,precio,total_usdt
999,0.9380,4006
998,0.9379,15
997,0.9374,22
996,0.9370,41
995,0.9369,33
...,...,...
4,0.7219,6855
3,0.7218,2868
2,0.7217,591
1,0.7216,297


In [105]:
compras = order_book['bids']
compras = pd.DataFrame(compras, columns=['precio','total_usdt'])
compras

,precio,total_usdt
0,0.7212,1038
1,0.7211,4706
2,0.7210,2774
3,0.7209,4372
4,0.7208,8334
...,...,...
995,0.5680,3521
996,0.5679,18
997,0.5674,78
998,0.5673,35


## b. Obtención de rangos de precios

- Para obtener los rangos y obtener los puntos a cómo lo hace el gafas, debemos crear rangos
- Para crear los rangos necesitamos el precio mínimo y máximo para redondearlos hacia abajo y arriba respecto a los múltiplos de los libros

### i. Obtención de máximos y mínimos precios

- Para poder hacer esto, tenemos que decirle al Dataframe qué tipo de valores son las columnas
- Esto es porque, como se puede observar a continuación, los marca como object

In [106]:
compras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   precio      1000 non-null   object
 1   total_usdt  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


- Para modificar el tipo de objetos de las columnas, hacemos:

In [107]:
ventas[['precio','total_usdt']] = ventas[['precio','total_usdt']].astype('float')
ventas

,precio,total_usdt
999,0.9380,4006.0
998,0.9379,15.0
997,0.9374,22.0
996,0.9370,41.0
995,0.9369,33.0
...,...,...
4,0.7219,6855.0
3,0.7218,2868.0
2,0.7217,591.0
1,0.7216,297.0


In [108]:
compras[['precio','total_usdt']] = compras[['precio','total_usdt']].astype('float')
compras

,precio,total_usdt
0,0.7212,1038.0
1,0.7211,4706.0
2,0.7210,2774.0
3,0.7209,4372.0
4,0.7208,8334.0
...,...,...
995,0.5680,3521.0
996,0.5679,18.0
997,0.5674,78.0
998,0.5673,35.0


- Ahora sí, obtenemos los máximos y mínimos

In [109]:
max_ventas = ventas['precio'].max()
min_ventas = ventas['precio'].min()
print('Ventas(mín-máx): {} - {}'.format(min_ventas, max_ventas))

max_compras = compras['precio'].max()
min_compras = compras['precio'].min()
print('Compras(mín-máx): {} - {}'.format(min_compras, max_compras))

Ventas(mín-máx): 0.7215 - 0.938
Compras(mín-máx): 0.5671 - 0.7212


### ii. Obtención de Dataframes con los Rangos

- Para esto, necesitamos tener los múltiplos de las moendas cómo lo hace el gafas
- Desafortunadamante, aún no encuentro cómo obtener los múltiplos de binance por lo que toca escribirlos en el archivo coin_data.py
- Para obtener el inicio de inicio y final de un rango, hacemos:

1. (precio_minimo / multiplo) ; donde el múltiplo es el que obtenemos de coind_data en i1
2. Luego redondeamos al entero inferior con math.floor()
3. Y multiplicamos por el múltiplo nuevamente para obtener el inicio del rango
4. (precio_maximo / multiplo)
5. redondeamos hacia el entero superior con math.ceil()
6. Y multiplicamos por el múltiplo nuevamente para obtener el máximo del rango

#### Obtención de valores iniciales y finales de rango con primer múltiplo

In [110]:
# Ventas
inicio_rango_ventas_1 = ( math.floor( (min_ventas / data[ticker]['i1']) ) ) * data[ticker]['i1']
final_rango_ventas_1 = ( math.ceil( (max_ventas / data[ticker]['i1']) ) ) * data[ticker]['i1']
print('Rango Ventas: {} - {}'.format(inicio_rango_ventas_1,final_rango_ventas_1))
# Compras
inicio_rango_compras_1 = ( math.floor( (min_compras / data[ticker]['i1']) ) ) * data[ticker]['i1']
final_rango_compras_1 = ( math.ceil( (max_compras / data[ticker]['i1']) ) ) * data[ticker]['i1']
print('Rango Compras: {} - {}'.format(inicio_rango_compras_1,final_rango_compras_1))

Rango Ventas: 0.72 - 0.9400000000000001
Rango Compras: 0.56 - 0.73


#### Obtención de valores iniciales y finales de rango con segundo múltiplo

In [111]:
# Ventas
inicio_rango_ventas_2 = ( math.floor( (min_ventas / data[ticker]['i2']) ) ) * data[ticker]['i2']
final_rango_ventas_2 = ( math.ceil( (max_ventas / data[ticker]['i2']) ) ) * data[ticker]['i2']
print('Rango Ventas: {} - {}'.format(inicio_rango_ventas_2,final_rango_ventas_2))
# Compras
inicio_rango_compras_2 = ( math.floor( (min_compras / data[ticker]['i2']) ) ) * data[ticker]['i2']
final_rango_compras_2 = ( math.ceil( (max_compras / data[ticker]['i2']) ) ) * data[ticker]['i2']
print('Rango Compras: {} - {}'.format(inicio_rango_compras_2,final_rango_compras_2))

Rango Ventas: 0.721 - 0.9380000000000001
Rango Compras: 0.5670000000000001 - 0.722


### iii. Obtención de los dataframes agrupados por rangos

- Ya que tenemos los valores inicial y final de los rangos, creamos el Dataframe con datos agrupados por rangos
- Solo que, antes tenemos que tener un rango con incrementos de acuerdo al múltiplo. Para esto usamos np.arange()

- Con múltiplo 1

In [112]:
rango_v_1 = np.arange(inicio_rango_ventas_1, final_rango_ventas_1+data[ticker]['i1'], data[ticker]['i1'])
ventas_ag_1 = ventas.groupby(pd.cut(ventas.precio,rango_v_1)).sum()
print('Ventas agrupadas segun incremento 1')
print(rango_v_1)
print(ventas_ag_1.sort_index(ascending=False))
rango_c_1 = np.arange(inicio_rango_compras_1, final_rango_compras_1+data[ticker]['i1'], data[ticker]['i1'])
compras_ag_1 = compras.groupby(pd.cut(compras.precio,rango_c_1)).sum()
print('\nCompras agrupadas segun incremento 1')
print(rango_c_1)
print(compras_ag_1.sort_index())

Ventas agrupadas segun incremento 1
[0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85
 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95]
               precio  total_usdt
precio                           
(0.94, 0.95]   0.0000         0.0
(0.93, 0.94]  21.4834     15834.0
(0.92, 0.93]  17.5730     34585.0
(0.91, 0.92]  26.5319     17016.0
(0.9, 0.91]   17.2068     29510.0
(0.89, 0.9]   25.9795     26538.0
(0.88, 0.89]  23.0377     15182.0
(0.87, 0.88]  23.6375     26775.0
(0.86, 0.87]  20.7731     19326.0
(0.85, 0.86]  23.0845     16875.0
(0.84, 0.85]  33.8314    468117.0
(0.83, 0.84]  33.4043    456408.0
(0.82, 0.83]  33.0142     47969.0
(0.81, 0.82]  26.9150     54250.0
(0.8, 0.81]   44.2810     56566.0
(0.79, 0.8]   47.7213     58022.0
(0.78, 0.79]  49.4612    117567.0
(0.77, 0.78]  48.8385    154976.0
(0.76, 0.77]  51.2460    185020.0
(0.75, 0.76]  58.1281    162094.0
(0.74, 0.75]  58.0902    350951.0
(0.73, 0.74]  57.3374    224832.0
(0.72, 0.73]  60.2322    

- con múltiplo 2

In [113]:
rango_v_2 = np.arange(inicio_rango_ventas_2, final_rango_ventas_2+data[ticker]['i2'], data[ticker]['i2'])
ventas_ag_2 = ventas.groupby(pd.cut(ventas.precio,rango_v_2)).sum()
print('Ventas agrupadas segun incremento 2')
#print(rango_v_2)
print(ventas_ag_2.sort_index(ascending=False))
rango_c_2 = np.arange(inicio_rango_compras_2, final_rango_compras_2+data[ticker]['i2'], data[ticker]['i2'])
compras_ag_2 = compras.groupby(pd.cut(compras.precio,rango_c_2)).sum()
print('\nCompras agrupadas segun incremento 2')
#print(rango_c_2)
print(compras_ag_2.sort_index(ascending=False))

Ventas agrupadas segun incremento 2
                precio  total_usdt
precio                            
(0.938, 0.939]  0.0000         0.0
(0.937, 0.938]  2.8133      4043.0
(0.936, 0.937]  2.8104        85.0
(0.935, 0.936]  1.8710        34.0
(0.934, 0.935]  1.8691        75.0
...                ...         ...
(0.725, 0.726]  6.5297     53152.0
(0.724, 0.725]  7.2455    132443.0
(0.723, 0.724]  7.2355    124190.0
(0.722, 0.723]  7.2255     71020.0
(0.721, 0.722]  4.3305     16607.0

[218 rows x 2 columns]

Compras agrupadas segun incremento 2
                precio  total_usdt
precio                            
(0.721, 0.722]  1.4423      5744.0
(0.72, 0.721]   7.2055     58261.0
(0.719, 0.72]   7.1955    100610.0
(0.718, 0.719]  7.1855    111700.0
(0.717, 0.718]  7.1755    117556.0
...                ...         ...
(0.571, 0.572]  1.7152       167.0
(0.57, 0.571]   1.7114      5483.0
(0.569, 0.57]   2.8478     14248.0
(0.568, 0.569]  3.9795       689.0
(0.567, 0.568]  2.8377     

#### Obtención de los shock points

- Ahora, para obtener los shock point, necesitamos obtener el dato donde el total usdt es máximo
- Traer el índice y extraer:
    - Para ventas: el lado derecho del intervalo. Porque es la parte superior del intervalo
    - Para compras: el lado izquiero del intervalo. Porque es la parte inferior del intervalo

In [114]:
from numpy import sort


sp_v_1 = ventas_ag_1['total_usdt'].idxmax().right
#print(sp_v_1)
#print(ventas_ag_2)
#print(ventas_ag_2['total_usdt'].max())
#print(ventas_ag_2['total_usdt'].idxmax())
#print(ventas_ag_2['total_usdt'].idxmax())
sp_v_2 = ventas_ag_2['total_usdt'].idxmax().right
sp_v = []
sp_v.append(sp_v_1)
sp_v.append(sp_v_2)
sp_v = sort(sp_v)
sp_v_1 = sp_v[1]
sp_v_2 = sp_v[0]
print(f'Ventas:\n***************\n2.  {sp_v_1}\n1.  {sp_v_2}')

Ventas:
***************
2.  0.845
1.  0.73


In [115]:
sp_c_1 = compras_ag_1['total_usdt'].idxmax().left
#print(sp_c_1)
#print(compras_ag_2)
#print(compras_ag_2['total_usdt'].max())
#print(compras_ag_2['total_usdt'].idxmax())
sp_c_2 = compras_ag_2['total_usdt'].idxmax().left
#print(sp_c_2)
sp_c = []
sp_c.append(sp_c_1)
sp_c.append(sp_c_2)
sp_c = sort(sp_c)
sp_c_1 = sp_c[1]
sp_c_2 = sp_c[0]
print(f'Compras:\n***************\n1.  {sp_c_1}\n2.  {sp_c_2}')

Compras:
***************
1.  0.62
2.  0.62


# 3. Analizar Shock Points

- El análisis de los puntos se hará de la siguiente manera:
    - Para ventas, obtenemos la distancia o porcentaje entre puntos de venta: pct_v = ( (venta_fin - venta_inicio)/venta_din ) * 100
    - Para compras, lo calculasmo así: pct_c = ( (compra_inicio - compra_fin)/compra_inicio ) * 100
- Luego, debemos obtener la distancia entre el primer punto de venta y de compras, en este caso tenemos una ligera variación por lo que obtendremos el promedio. 
    - (1)=  (compras_inicio - ventas_inicio)/compras_inicio ) * -100
    - (2)=  (ventas_inicio - compras_inicio)/ventas_inicio ) * 100
    - pct_sep = promedio(1,2)

In [124]:
pct_ventas = round( ( (sp_v_1 - sp_v_2) / sp_v_2 ) * 100, 2 )
print(f'Distancia entre ventas: {pct_ventas}%')

pct_compras = round( ( (sp_c_1 - sp_c_2) / sp_c_1 ) * 100, 2)
print(f'Distancia entre compraas: {pct_compras}%')
print(sp_c_1, ', ',sp_c_2)
pct_sep_1 = (( (sp_c_1 - sp_v_2) / sp_c_1 ) * -100)
pct_sep_2 = ((sp_v_2 - sp_c_1) / sp_v_2 ) * 100
pct_sep = round((pct_sep_1+pct_sep_2)/2,2)
print(f'Porcentaje entre compra y venta: {pct_sep}%')

Distancia entre ventas: 15.75%
Distancia entre compraas: 0.0%
0.62 ,  0.62
Porcentaje entre compra y venta: 16.41%


# 4. Resultado del Análisis de los Shock Points

- Una vez que tenemos las distancias, procedemos a comparar lo siguiente:
    - Para compras:
        - Distancia entre compra y venta debe ser mayor a la distancia entre compras por 2.5 o más veces
    - Para ventas:
        - - Distancia entre compra y venta debe ser mayor a la distancia entre ventas por 2.5 o más veces
- Si se cumplen esas condiciones enviamos señal para compra y/o venta
- Una nota aquí es que, a la distancia entre ventas o entre compras, habría que aumentarle la distancia entre el segundo shock_point y el SL

In [123]:
ratio = 2.5
flag_compra = False
flag_venta = False

print(pct_sep)
print(pct_compras)

if (pct_sep/pct_ventas) >= ratio: # Aquí es pct_sep/(pct_ventas+SL)
    print(f'Se puede vender en {ticker}')
    flag_venta = True
if (pct_sep/pct_compras) >= ratio: # Aquí es pct_sep/(pct_compras+SL)
    print(f'Se puede comprar en {ticker}')
    flag_compra = True
if not flag_compra and not flag_venta:
    print('Nada que hacer, sigue buscando')


16.41
0.0
Se puede comprar en STORJUSDT


/tmp/ipykernel_697/3740063611.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  if (pct_sep/pct_compras) >= ratio: # Aquí es pct_sep/(pct_compras+SL)


# 5. Calcular SL, TP y Recompras

- En el caso que sí se habilite alguna operación debemos tener una gestión por moneda dentro del archivo coin_data para determinar qué tanta separación entre el segundo shock point y nuestro SL debe haber
- Esto lo veremos de acuerdo a nuestro análisis de nuestras monedas teniendo uno por default o uno por moneda

## Entrada

In [118]:
if flag_venta:
    print(f'Entrada Venta:\t${sp_v_2}')

if flag_compra:
    print(f'Entrada Compra:\t${sp_c_1}')

Entrada Compra:	$0.62


## SL

- Primero debemos traer el balance de nuestra cuenta de futuros

In [119]:
import os

In [120]:
#x = 0
# while True:
#    if x < 100:
#        actual_price = client.futures_mark_price(symbol=ticker)
#        os.system('clear')
#        print('Precio actual {}'.format(actual_price))
#        x += 1


## Recompras / Reventas

- Para las recompras usaremos la misma configuración del gafas:
    - Compras a cada 2%
    - Comprando el 40% de las monedas que esten al momento
    - SL del 10% del capital

# 6. Calcular recompras

# 7. Ejecutar órdenes